In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#load_n = 50 # from 280
X_train = np.load('data/X_train.npy')#[0:load_n]
y_train = np.genfromtxt('data/y_1.csv', delimiter='\n')#[0:load_n]


In [11]:
# Add feature with random numbers
# gen random numbers
# add to data set
max_int = np.max(X_train[0])
X_train_new = []
print(len(X_train[0]))
for i, row in enumerate(X_train):
    X_train_new.append(np.append(X_train[i], np.random.randint(max_int)))
    
X_train = np.array(X_train_new)
del X_train_new
print(len(X_train[0]))

# Do it like this
tN = 10
a = np.random.rand(N,N)
b = np.zeros((N,N+1))
b[:,:-1] = a

6443008


In [13]:
# Train decision tree model
from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import ExtraTreesClassifier

regr = DecisionTreeRegressor(max_depth=4)
regr.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [30]:
# Show which features are important incl. the random number feature

importances = regr.feature_importances_

data = zip(range(0, len(X_train[0])), importances)
sorted_by_second = sorted(data, key=lambda tup: tup[1], reverse=True)

# Random column seems to have no importance, which is good
# But only few columns are important, which is strange
sorted_by_second[:30]

[(2840202, 0.72266944714206394),
 (3068332, 0.06888752125207484),
 (3760490, 0.065983129116628764),
 (3216672, 0.043998933908273405),
 (3207918, 0.024494639376667009),
 (5364180, 0.016905535866513416),
 (1149200, 0.012810222014025385),
 (1733846, 0.012042646594167332),
 (3529729, 0.011361716114977936),
 (1891425, 0.006949498949287862),
 (4474482, 0.0059437029132285938),
 (3778271, 0.0039435903799835501),
 (3788513, 0.0027202129813155328),
 (1361955, 0.0012102506893728847),
 (4455500, 7.895270141953137e-05),
 (0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0)]

In [34]:
# Save which features are unimportant
indexes = [x for x, y in sorted_by_second[:1000] if y > 0]
np.savetxt('decision_tree_features.csv', indexes, delimiter=',', fmt='%d') 

In [62]:
# SVM with rbf kernel
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=10000, learning_rate=0.1)

test_data = np.load('data/X_test.npy')
X_new = []
X_test = []
for row in X_train:
    X_new.append(np.take(row, indexes))
for row in test_data:
    X_test.append(np.take(row, indexes))


model.fit(X_new, y_train)
predictions = model.predict(X_test)
f = open('out.csv', 'w')
f.write('ID,Prediction\n')
for i, p in enumerate(predictions):
    f.write(str(i+1) + ',' + str(p) + '\n')
f.close()
predictions

array([ 73.40512459,  74.5697127 ,  58.38429152,  72.76562466,
        81.06703432,  71.74375949,  33.41941942,  36.67235924,
        40.20867067,  56.77195744,  23.90850339,  52.92033455,
        76.31552397,  53.73391486,  22.2951098 ,  78.96117731,
        20.01342389,  80.84705513,  84.50014045,  90.14391054,
        79.25350692,  25.02058741,  77.08701493,  83.45518921,
        77.78430604,  20.17269018,  62.02328321,  25.29163087,
        40.23215908,  43.70675869,  68.26578925,  31.52213706,
        24.01946585,  50.81632797,  40.56532801,  83.69276131,
        87.42179774,  85.7516236 ,  72.70729284,  83.9147481 ,
        25.03122095,  74.00605105,  38.6955044 ,  22.35470433,
        53.56902238,  55.98624594,  42.58009366,  22.46893444,
        25.84916212,  79.84550255,  46.93439649,  75.25405342,
        38.89095892,  35.80934371,  52.64421075,  81.41789278,
        75.86817922,  76.43871159,  33.6248466 ,  69.89181464,
        71.21289036,  70.54314302,  32.17123687,  33.85

In [63]:
np.mean(model.train_score_)

0.56009773790282791

Obviously this way didn't work out. Score was ~230. :)